In [2]:
import tensorflow as tf

Os dados MNIST estão disponíveis com o Keras. Primeiro, importe o **tensorflow**. Em seguida, defina algumas constantes, como o **batch_size**, **no_classes** e **epochs**. O **batch_size** pode ser selecionado com base na RAM disponível em sua máquina. Quanto maior o valor de **batch_size**, mais RAM é necessária. O impacto do **batch_size** na precisão é mínima. **no_classes** é igual a 10 aqui e será diferente para problemas diferentes. **epochs** determina quantas vezes o treinamento precisa passar pelo conjunto de dados completo. Se a perda for reduzida no final de todas as épocas, pode ser definida como um número alto. Em alguns casos, o treinamento por mais tempo pode dar uma melhor precisão. Vamos agora olhar para as etapas para criar o conjunto de dados:

Defina as dimensões das imagens de entrada, assim como segue:

In [3]:
batch_size = 128
no_classes = 10
epochs = 2
image_height, image_width = 28, 28

In [4]:
# Load the data from disk to memory using the Keras utilities:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
# Reshape the vector into an image format, and define the input dimension for the convolution 
# using the code given:
x_train = x_train.reshape(x_train.shape[0], image_height, image_width, 1)
x_test = x_test.reshape(x_test.shape[0], image_height, image_width, 1)
input_shape = (image_height, image_width, 1)

In [6]:
# Convert the data type to float as follows
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
# Normalize the data by subtracting the mean of the data:
x_train /= 255
x_test /= 255

In [8]:
# Convert the categorical labels to one-shot encoding:
y_train = tf.keras.utils.to_categorical(y_train, no_classes)
y_test = tf.keras.utils.to_categorical(y_test, no_classes)

Nesta seção, usaremos algumas camadas de convolução seguidas por camadas totalmente conectadas para treinar o conjunto de dados anterior. Construa um modelo seqüencial simples com duas camadas de convolução seguidas por camadas de agrupamento, dropout e densa. Um modelo sequencial tem o método add para empilhar camadas uma acima da outra. A primeira camada tem 64 filtros e a segunda camada tem 128 filtros. O tamanho do kernel é 3 para todos os filtros. Aplique o **pooling** máximo após as camadas de convolução. A saída das camadas de convolução é achatada conectando-se a algumas camadas totalmente conectadas com conexões de dropout.

A última camada está conectada ao softmax, pois esse é um problema de classificação multiclasse. O código a seguir mostra como definir o modelo:

In [9]:
def simple_cnn(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

simple_cnn_model = simple_cnn(input_shape)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


O modelo é apenas definido e precisa ser compilado. Durante a perda(loss) de compilação, o otimizador e as métricas precisam ser definidos. A perda(loss) será entropia cruzada, otimizada pelo algoritmo Adam, e reportaremos a precisão como a métrica. Usando os dados carregados, treine e avalie os dados. Carregue os dados de treinamento com os parâmetros de treinamento e ajuste o modelo:

In [10]:
simple_cnn_model.fit(x_train, y_train, batch_size, epochs, (x_test, y_test))
train_loss, train_accuracy = simple_cnn_model.evaluate(
    x_train, y_train, verbose=0)

print('Train data loss:', train_loss)
print('Train data accuracy:', train_accuracy)

Epoch 1/2
Epoch 2/2
('Train data loss:', 0.019273758619398965)
('Train data accuracy:', 0.99478333333333335)


In [11]:
test_loss, test_accuracy = simple_cnn_model.evaluate(
    x_test, y_test, verbose=0)
print('Test data loss:', test_loss)
print('Test data accuracy:', test_accuracy)

('Test data loss:', 0.029745608770661056)
('Test data accuracy:', 0.99139999999999995)


Loss for train data: 0.019273758619398965<br/>
Accuracy of train data: 0.99478333333333335<br/>
Loss for test data: 0.029745608770661056<br/>
Accuracy of test data: 0.99139999999999995<br/>